In [36]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor,execute_values
import numpy as np
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables (optional, for security)
load_dotenv()



True

In [37]:


DB_CONFIG = {
     'host': os.getenv('DB_HOST'),
     'database': os.getenv('DB_NAME'),
     'user': os.getenv('DB_USER'),
     'password': os.getenv('DB_PASSWORD'),
     'port': os.getenv('DB_PORT'),
     }




In [38]:
def connect_to_database():
    """Create database connection"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        connection.autocommit = True
        print("✅ Successfully connected to PostgreSQL database")
        return connection
    except Exception as e:
        print(f"❌ Error connecting to database: {e}")
        return None

## vehiculos

In [30]:
# === 1. Read CSV ===
df = pd.read_csv("../data/df_vehiculos_procesado_kw.csv", sep=",", decimal=".", 
                 na_values=["", " ", "Sin clasificación"])

# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns to match DB ===
df.rename(columns={
    "Matricula": "matricula",
    "Modelo": "Modelo",  # Keep as Modelo for now, will use it to split
    "Clasificación Energética": "etiqueta",
    "Consumo MIN": "consumo_min",
    "Consumo MAX": "consumo_max",
    "Emisiones MIN (gCO2/km)": "emisiones_min",
    "Emisiones MAX (gCO2/km)": "emisiones_max",
    "Motorización": "motorizacion",  # This was missing!
    "km": "km_original",  # Rename to avoid conflict later
    "kw_minimo": "kw_minimo",
    "kw_maximo": "kw_maximo",
}, inplace=True)

print("📌 Columns after renaming:", df.columns.tolist())

# === 3. Split brand and model ===
df["marca"] = df["Modelo"].apply(lambda x: str(x).split()[0] if pd.notnull(x) else None)
df["modelo"] = df["Modelo"].apply(lambda x: " ".join(str(x).split()[1:]) if pd.notnull(x) and len(str(x).split()) > 1 else None)

# === 4. Add missing DB columns ===
df["tipo"] = df["motorizacion"].apply(
    lambda x: "eléctrico" if "Eléctrico" in str(x) or "eléctrico" in str(x).lower() else None
)
df["km"] = None  # Set km to None as required by DB schema

# === 5. Reorder to match DB schema (without id_usuario) ===
columns = ["matricula", "marca", "modelo", "etiqueta", "tipo",
           "consumo_min", "consumo_max", "emisiones_min", "emisiones_max",
           "motorizacion", "km", "kw_minimo", "kw_maximo"]

# Check if all required columns exist
missing_columns = [col for col in columns if col not in df.columns]
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    for col in missing_columns:
        df[col] = None

records = df[columns].where(pd.notnull(df), None).values.tolist()  # Replace NaN → None

print(f"📌 Prepared {len(records)} records for insertion")

# === 6. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    try:
        with connection.cursor() as cur:
            insert_query = """
                INSERT INTO vehiculo (
                    matricula, marca, modelo, etiqueta, tipo, 
                    consumo_min, consumo_max, emisiones_min, emisiones_max,
                    motorizacion, km, kw_minimo, kw_maximo
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT (matricula) DO NOTHING;
            """
            
            successful_inserts = 0
            for i, row in enumerate(records):
                try:
                    cur.execute(insert_query, row)
                    successful_inserts += 1
                except Exception as e:
                    print(f"❌ Error inserting row {i}: {e}")
                    print(f"   Row data: {row}")
            
        connection.commit()
        print(f"✅ Successfully inserted {successful_inserts} out of {len(records)} rows into vehiculo table")
        
    except Exception as e:
        print(f"❌ Database operation failed: {e}")
        connection.rollback()
    finally:
        connection.close()
else:
    print("❌ Could not establish database connection")

📌 CSV Columns Detected: ['Matricula', 'Modelo', 'Clasificación Energética', 'Consumo MIN', 'Consumo MAX', 'Emisiones MIN (gCO2/km)', 'Emisiones MAX (gCO2/km)', 'Motorización', 'km', 'kw_minimo', 'kw_maximo']
📌 Columns after renaming: ['matricula', 'Modelo', 'etiqueta', 'consumo_min', 'consumo_max', 'emisiones_min', 'emisiones_max', 'motorizacion', 'km_original', 'kw_minimo', 'kw_maximo']
📌 Prepared 22203 records for insertion
✅ Successfully connected to PostgreSQL database
✅ Successfully inserted 22203 out of 22203 rows into vehiculo table


## usuarios

In [16]:
# === 1. Read CSV ===
df = pd.read_csv("../data/usuarios.csv", sep=",", decimal=".", na_values=["", " ", "NULL"])
# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns to match DB ===
df.rename(columns={
    "id_usuario": "id_usuario",  # Keep but won't use in insert
    "usuario": "username",       # Map to DB column name
    "contraseña": "password",    # Spanish to English (with ñ)
    "email": "email",
    "nombre": "nombre", 
    "apellido": "apellido",
    "role": "role",
    "active": "active"
}, inplace=True)

# === 3. Data processing (if needed) ===
# Convert 'true'/'false' strings to boolean if needed
df["active"] = df["active"].apply(
    lambda x: True if str(x).lower() == "true" else False if str(x).lower() == "false" else x
)

# === 4. Add missing DB columns (if any) ===
# All required columns are already present in CSV

# === 5. Reorder to match DB schema (without id_usuario for auto-increment) ===
columns = ["username", "password", "email", "nombre", "apellido", "role", "active"]
records = df[columns].where(pd.notnull(df), None).values.tolist()  # Replace NaN → None

# === 6. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    with connection.cursor() as cur:
        insert_query = """
            INSERT INTO usuario (
                username, password, email, nombre, apellido, role, active
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s
            );
        """
        inserted_count = 0
        for i, row in enumerate(records):
            try:
                cur.execute(insert_query, row)
                inserted_count += 1
            except Exception as e:
                print(f"❌ Error inserting row {i+1}: {e}")
                print(f"   Data: {row}")
    connection.commit()
    print(f"✅ Inserted {inserted_count} out of {len(records)} rows into usuario")
    connection.close()

📌 CSV Columns Detected: ['id_usuario', 'usuario', 'contraseña', 'email', 'nombre', 'apellido', 'role', 'active']
✅ Successfully connected to PostgreSQL database
✅ Inserted 12 out of 12 rows into usuario


## Rutas

In [15]:
# === 1. Read CSV ===
df = pd.read_csv("../data/rutas.csv", sep=",", decimal=".", na_values=["", " ", "NULL"])
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns (not strictly needed if CSV already matches DB) ===
df.rename(columns={
    "matricula": "matricula",
    "polyline": "polyline",
    "paradas": "paradas",
    "total_km": "total_km",
    "fecha_inicio": "fecha_inicio",
    "fecha_fin": "fecha_fin",
    "tiempo_total": "tiempo_total",
    "tiempos_paradas": "tiempos_paradas",
    "kms_paradas": "kms_paradas"
}, inplace=True)

# === 3. Data processing (optional) ===
# No "active" column needed here, remove that part

# === 4. Reorder columns to match DB schema ===
columns = [
    "matricula", "polyline", "paradas", "total_km", 
    "fecha_inicio", "fecha_fin", "tiempo_total",
    "tiempos_paradas", "kms_paradas"
]
records = df[columns].where(pd.notnull(df), None).values.tolist()

# === 5. Insert into PostgreSQL ===
# === 5. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    try:
        with connection.cursor() as cur:
            insert_query = """
                INSERT INTO ruta (
                    matricula, polyline, paradas, total_km, 
                    fecha_inicio, fecha_fin, tiempo_total,
                    tiempos_paradas, kms_paradas
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s
                );
            """
            
            # Use executemany for batch insert
            cur.executemany(insert_query, records)
            connection.commit()
            print(f"✅ Inserted {len(records)} rows into ruta")
    
    except Exception as e:
        print(f"❌ Transaction failed: {e}")
        connection.rollback()
    
    finally:
        connection.close()

📌 CSV Columns Detected: ['matricula', 'polyline', 'paradas', 'total_km', 'fecha_inicio', 'fecha_fin', 'tiempo_total', 'tiempos_paradas', 'kms_paradas']
✅ Successfully connected to PostgreSQL database
✅ Inserted 5 rows into ruta


## Tipo combustible y precio  

In [41]:
# === 1. Read CSV ===
df = pd.read_csv("../data/df_combustible_precios_euskadi_2022-2025.csv", sep=",", decimal=".", 
                 na_values=["", " ", "Sin clasificación"])

# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns to match DB ===
df.rename(columns={
    "Fecha": "fecha",
    "C.P.": "codigo_postal",
    "Latitud": "latitud",
    "Longitud": "longitud",
    "Precio Biodiesel": "precio_biodiesel",
    "Precio Bioetanol": "precio_bioetanol",
    "Precio Gas Natural Comprimido": "precio_gas_natural_comprimido",
    "Precio Gas Natural Licuado": "precio_gas_natural_licuado",
    "Precio Gases licuados del petróleo": "precio_gases_licuados_petroleo",
    "Precio Gasoleo A": "precio_gasoleo_a",
    "Precio Gasoleo B": "precio_gasoleo_b",
    "Precio Gasoleo Premium": "precio_gasoleo_premium",
    "Precio Gasolina 95 E5": "precio_gasolina_95_e5",
    "Precio Gasolina 95 E5 Premium": "precio_gasolina_95_e5_premium",
    "Precio Gasolina 98 E5": "precio_gasolina_98_e5",
}, inplace=True)

print("📌 Columns after renaming:", df.columns.tolist())

# === 3. Data type conversions ===
# Convert fecha to datetime
if 'fecha' in df.columns:
    try:
        df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y %H:%M:%S')
        print("✅ Converted fecha to datetime")
    except Exception as e:
        print(f"⚠️ Could not convert fecha to datetime: {e}")

# Convert coordinates to numeric
coordinate_columns = ['latitud', 'longitud']
for col in coordinate_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except Exception as e:
            print(f"⚠️ Could not convert {col} to numeric: {e}")

# Convert postal code to integer
if 'codigo_postal' in df.columns:
    try:
        df['codigo_postal'] = pd.to_numeric(df['codigo_postal'], errors='coerce').astype('Int64')
        print("✅ Converted codigo_postal to integer")
    except Exception as e:
        print(f"⚠️ Could not convert codigo_postal: {e}")

# Convert price columns (handle European decimal format with comma)
price_columns = [
    'precio_biodiesel', 'precio_bioetanol', 'precio_gas_natural_comprimido',
    'precio_gas_natural_licuado', 'precio_gases_licuados_petroleo', 'precio_gasoleo_a',
    'precio_gasoleo_b', 'precio_gasoleo_premium', 'precio_gasolina_95_e5',
    'precio_gasolina_95_e5_premium', 'precio_gasolina_98_e5'
]

for col in price_columns:
    if col in df.columns:
        try:
            # Replace comma with dot for decimal conversion
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            print(f"✅ Converted {col} to numeric (comma to dot)")
        except Exception as e:
            print(f"⚠️ Could not convert {col}: {e}")

# === 4. Add existing DB columns that aren't in CSV ===
# Add any additional fields your fuel_prices table might need
# df["id_estacion"] = None  # If you have station IDs
# df["created_at"] = pd.Timestamp.now()  # If you track insertion time

# === 5. Define columns to match DB schema ===
columns = [
    "fecha", "codigo_postal", "latitud", "longitud",
    "precio_biodiesel", "precio_bioetanol", "precio_gas_natural_comprimido",
    "precio_gas_natural_licuado", "precio_gases_licuados_petroleo", "precio_gasoleo_a",
    "precio_gasoleo_b", "precio_gasoleo_premium", "precio_gasolina_95_e5",
    "precio_gasolina_95_e5_premium", "precio_gasolina_98_e5"
]

# Check if all required columns exist
missing_columns = [col for col in columns if col not in df.columns]
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    for col in missing_columns:
        df[col] = None

# Replace NaN with None for PostgreSQL compatibility
records = df[columns].where(pd.notnull(df[columns]), None).values.tolist()

print(f"📌 Prepared {len(records)} records for insertion")

# Show a sample of the data
if len(records) > 0:
    print("📌 Sample record:")
    sample_dict = dict(zip(columns, records[0]))
    for key, value in sample_dict.items():
        print(f"  {key}: {value}")

# === 6. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    try:
        with connection.cursor() as cur:
            insert_query = """
                INSERT INTO tipo_combustible (
                    fecha, codigo_postal, latitud, longitud,
                    precio_biodiesel, precio_bioetanol, precio_gas_natural_comprimido,
                    precio_gas_natural_licuado, precio_gases_licuados_petroleo, precio_gasoleo_a,
                    precio_gasoleo_b, precio_gasoleo_premium, precio_gasolina_95_e5,
                    precio_gasolina_95_e5_premium, precio_gasolina_98_e5
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT DO NOTHING;
            """
            
            successful_inserts = 0
            failed_inserts = 0
            
            for i, row in enumerate(records):
                try:
                    cur.execute(insert_query, row)
                    successful_inserts += 1
                except Exception as e:
                    failed_inserts += 1
                    print(f"❌ Error inserting row {i}: {e}")
                    print(f"   Row data: {row}")
                    # Print first few elements to avoid too much output
                    if i < 5:
                        sample_data = dict(zip(columns, row))
                        print(f"   Sample data: {sample_data}")
            
        connection.commit()
        print(f"✅ Successfully inserted {successful_inserts} out of {len(records)} rows into tipo_combustible table")
        if failed_inserts > 0:
            print(f"❌ Failed to insert {failed_inserts} rows")
        
    except Exception as e:
        print(f"❌ Database operation failed: {e}")
        connection.rollback()
    finally:
        connection.close()
else:
    print("❌ Could not establish database connection")

C:\Users\Dev\AppData\Local\Temp\ipykernel_19520\1979127395.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/df_combustible_precios_euskadi_2022-2025.csv", sep=",", decimal=".",


📌 CSV Columns Detected: ['Unnamed: 0', 'Fecha', 'C.P.', 'Latitud', 'Longitud', 'Precio Biodiesel', 'Precio Bioetanol', 'Precio Gas Natural Comprimido', 'Precio Gas Natural Licuado', 'Precio Gases licuados del petróleo', 'Precio Gasoleo A', 'Precio Gasoleo B', 'Precio Gasoleo Premium', 'Precio Gasolina 95 E5', 'Precio Gasolina 95 E5 Premium', 'Precio Gasolina 98 E5']
📌 Columns after renaming: ['Unnamed: 0', 'fecha', 'codigo_postal', 'latitud', 'longitud', 'precio_biodiesel', 'precio_bioetanol', 'precio_gas_natural_comprimido', 'precio_gas_natural_licuado', 'precio_gases_licuados_petroleo', 'precio_gasoleo_a', 'precio_gasoleo_b', 'precio_gasoleo_premium', 'precio_gasolina_95_e5', 'precio_gasolina_95_e5_premium', 'precio_gasolina_98_e5']
✅ Converted fecha to datetime
✅ Converted codigo_postal to integer
✅ Converted precio_biodiesel to numeric (comma to dot)
✅ Converted precio_bioetanol to numeric (comma to dot)
✅ Converted precio_gas_natural_comprimido to numeric (comma to dot)
✅ Converte

KeyboardInterrupt: 

## Ticket

In [32]:
# === 1. Read CSV ===
df = pd.read_csv("../data/tickets_combustible_euskadi_2022-2025.csv", sep=",", decimal=".", 
                 na_values=["", " ", "Sin clasificación"])

# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns to match DB ===
df.rename(columns={
    "Fecha": "fecha",
    "TipoCarburante": "tipo_carburante",
    "Precio por Litro": "precio_por_litro",
    "Coordenadas": "coordenadas",
    "LitrosCoche": "litros_coche",
    "LitrosBus": "litros_bus",
    "ImporteCoche_euros": "importe_coche_euros",
    "ImporteBus_euros": "importe_bus_euros",
    "Latitud": "latitud",
    "Longitud": "longitud",
}, inplace=True)

print("📌 Columns after renaming:", df.columns.tolist())

# === 3. Data type conversions ===
# Convert fecha to datetime if needed
if 'fecha' in df.columns:
    try:
        df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y %H:%M:%S')
        print("✅ Converted fecha to datetime")
    except Exception as e:
        print(f"⚠️ Could not convert fecha to datetime: {e}")

# Convert numeric columns
numeric_columns = ['precio_por_litro', 'litros_coche', 'litros_bus', 
                   'importe_coche_euros', 'importe_bus_euros', 'latitud', 'longitud']

for col in numeric_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except Exception as e:
            print(f"⚠️ Could not convert {col} to numeric: {e}")

# === 4. Add any missing DB columns if needed ===
# (Add any additional columns your ticket table requires that aren't in the CSV)
# For example, if you need an id_usuario or other foreign key:
# df["id_usuario"] = None  # or some default value

# === 5. Define columns to match DB schema ===
# Adjust this list based on your actual ticket table schema
columns = ["fecha", "tipo_carburante", "precio_por_litro", "coordenadas",
           "litros_coche", "litros_bus", "importe_coche_euros", "importe_bus_euros",
           "latitud", "longitud"]

# Check if all required columns exist
missing_columns = [col for col in columns if col not in df.columns]
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    for col in missing_columns:
        df[col] = None

# Replace NaN with None for PostgreSQL compatibility
records = df[columns].where(pd.notnull(df[columns]), None).values.tolist()

print(f"📌 Prepared {len(records)} records for insertion")

# Show a sample of the data
if len(records) > 0:
    print("📌 Sample record:")
    print(dict(zip(columns, records[0])))

# === 6. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    try:
        with connection.cursor() as cur:
            # Adjust this INSERT query based on your actual ticket table schema
            insert_query = """
                INSERT INTO ticket (
                    fecha, tipo_carburante, precio_por_litro, coordenadas,
                    litros_coche, litros_bus, importe_coche_euros, importe_bus_euros,
                    latitud, longitud
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT DO NOTHING;
            """
            
            successful_inserts = 0
            failed_inserts = 0
            
            for i, row in enumerate(records):
                try:
                    cur.execute(insert_query, row)
                    successful_inserts += 1
                except Exception as e:
                    failed_inserts += 1
                    print(f"❌ Error inserting row {i}: {e}")
                    print(f"   Row data: {row}")
            
        connection.commit()
        print(f"✅ Successfully inserted {successful_inserts} out of {len(records)} rows into ticket table")
        if failed_inserts > 0:
            print(f"❌ Failed to insert {failed_inserts} rows")
        
    except Exception as e:
        print(f"❌ Database operation failed: {e}")
        connection.rollback()
    finally:
        connection.close()
else:
    print("❌ Could not establish database connection")

📌 CSV Columns Detected: ['Fecha', 'TipoCarburante', 'Precio por Litro', 'Coordenadas', 'LitrosCoche', 'LitrosBus', 'ImporteCoche_euros', 'ImporteBus_euros', 'Latitud', 'Longitud']
📌 Columns after renaming: ['fecha', 'tipo_carburante', 'precio_por_litro', 'coordenadas', 'litros_coche', 'litros_bus', 'importe_coche_euros', 'importe_bus_euros', 'latitud', 'longitud']
✅ Converted fecha to datetime
📌 Prepared 366746 records for insertion
📌 Sample record:
{'fecha': Timestamp('2024-11-02 00:00:00'), 'tipo_carburante': 'Precio Gasoleo B', 'precio_por_litro': nan, 'coordenadas': '(43.330917, -1.819028)', 'litros_coche': 52, 'litros_bus': 1137, 'importe_coche_euros': nan, 'importe_bus_euros': nan, 'latitud': 43.330917, 'longitud': -1.819028}
✅ Successfully connected to PostgreSQL database
✅ Successfully inserted 366746 out of 366746 rows into ticket table
